In [3]:
!wget "https://download.microsoft.com/download/3/E/1/3E1C3F21-ECDB-4869-8368-6DEBA77B919F/kagglecatsanddogs_5340.zip"

--2022-10-04 17:48:34--  https://download.microsoft.com/download/3/E/1/3E1C3F21-ECDB-4869-8368-6DEBA77B919F/kagglecatsanddogs_5340.zip
Resolving download.microsoft.com (download.microsoft.com)... 23.77.173.59, 2600:141b:e800:148c::317f, 2600:141b:e800:1483::317f
Connecting to download.microsoft.com (download.microsoft.com)|23.77.173.59|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 824887076 (787M) [application/octet-stream]
Saving to: ‘kagglecatsanddogs_5340.zip’

kagglecatsanddogs_5 100%[===================>] 786.67M   250MB/s    in 3.1s    

2022-10-04 17:48:37 (250 MB/s) - ‘kagglecatsanddogs_5340.zip’ saved [824887076/824887076]



In [4]:
!unzip -q "kagglecatsanddogs_5340.zip"

In [5]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import pathlib
import os
from tqdm import tqdm

In [6]:
len(os.listdir("PetImages/Dog"))

12501

In [7]:
len(os.listdir("PetImages/Cat"))

12501

In [8]:
DataPath = pathlib.Path("PetImages")
all_paths = DataPath.glob("*/*.jpg")
all_paths 

<generator object Path.glob at 0x7fce2ef6e6d0>

In [9]:
all_paths = list(all_paths)
all_paths[:10]

[PosixPath('PetImages/Cat/10997.jpg'),
 PosixPath('PetImages/Cat/4640.jpg'),
 PosixPath('PetImages/Cat/11492.jpg'),
 PosixPath('PetImages/Cat/6802.jpg'),
 PosixPath('PetImages/Cat/5920.jpg'),
 PosixPath('PetImages/Cat/6759.jpg'),
 PosixPath('PetImages/Cat/4731.jpg'),
 PosixPath('PetImages/Cat/585.jpg'),
 PosixPath('PetImages/Cat/7675.jpg'),
 PosixPath('PetImages/Cat/4788.jpg')]

In [10]:
all_paths = list(map(lambda x: str(x), all_paths))
all_paths[:10]

['PetImages/Cat/10997.jpg',
 'PetImages/Cat/4640.jpg',
 'PetImages/Cat/11492.jpg',
 'PetImages/Cat/6802.jpg',
 'PetImages/Cat/5920.jpg',
 'PetImages/Cat/6759.jpg',
 'PetImages/Cat/4731.jpg',
 'PetImages/Cat/585.jpg',
 'PetImages/Cat/7675.jpg',
 'PetImages/Cat/4788.jpg']

In [11]:
from random import shuffle

shuffle(all_paths)
all_paths[:10]

['PetImages/Dog/3645.jpg',
 'PetImages/Cat/12389.jpg',
 'PetImages/Dog/2061.jpg',
 'PetImages/Dog/5055.jpg',
 'PetImages/Cat/8776.jpg',
 'PetImages/Cat/1392.jpg',
 'PetImages/Cat/11096.jpg',
 'PetImages/Cat/2249.jpg',
 'PetImages/Dog/2727.jpg',
 'PetImages/Cat/7263.jpg']

In [12]:
def TestImageQuality(all_paths):
  new_all_paths = []
  for path in tqdm(all_paths):
    try:
      image = tf.io.read_file(path)
      image = tf.io.decode_jpeg(image, channels = 3)
    except:
      continue
    new_all_paths.append(path)
  return new_all_paths

all_paths = TestImageQuality(all_paths)
all_paths[:10]

100%|██████████| 25000/25000 [00:35<00:00, 710.15it/s]


['PetImages/Dog/3645.jpg',
 'PetImages/Cat/12389.jpg',
 'PetImages/Dog/2061.jpg',
 'PetImages/Dog/5055.jpg',
 'PetImages/Cat/8776.jpg',
 'PetImages/Cat/1392.jpg',
 'PetImages/Cat/11096.jpg',
 'PetImages/Cat/2249.jpg',
 'PetImages/Dog/2727.jpg',
 'PetImages/Cat/7263.jpg']

In [13]:
# Getting their respective labels
def get_label(image_path):
  return image_path.split("/")[-2]
all_labels = list(map(lambda x: get_label(x), all_paths))
all_labels[:10]

['Dog', 'Cat', 'Dog', 'Dog', 'Cat', 'Cat', 'Cat', 'Cat', 'Dog', 'Cat']

In [14]:
from sklearn.preprocessing import LabelEncoder

Le = LabelEncoder()
all_labels = Le.fit_transform(all_labels)

all_labels[:10]

array([1, 0, 1, 1, 0, 0, 0, 0, 1, 0])

In [15]:
from sklearn.model_selection import train_test_split
Train_paths, Val_paths, Train_labels, Val_labels = train_test_split(all_paths, all_labels)

In [16]:
# Function used to decode jpeg into tensor
def load(image, label):
  image = tf.io.read_file(image)
  image = tf.io.decode_jpeg(image, channels = 3)
  return image, label

In [17]:
Train_paths[:10], Train_labels[:10]

(['PetImages/Cat/10008.jpg',
  'PetImages/Dog/7787.jpg',
  'PetImages/Dog/5742.jpg',
  'PetImages/Dog/3921.jpg',
  'PetImages/Dog/11198.jpg',
  'PetImages/Cat/10183.jpg',
  'PetImages/Dog/2529.jpg',
  'PetImages/Cat/7932.jpg',
  'PetImages/Dog/12483.jpg',
  'PetImages/Dog/4539.jpg'],
 array([0, 1, 1, 1, 1, 0, 1, 0, 1, 1]))

In [18]:
Val_paths[:10], Val_labels[:10]

(['PetImages/Cat/583.jpg',
  'PetImages/Dog/3292.jpg',
  'PetImages/Dog/10506.jpg',
  'PetImages/Dog/788.jpg',
  'PetImages/Cat/6510.jpg',
  'PetImages/Dog/3910.jpg',
  'PetImages/Cat/2515.jpg',
  'PetImages/Cat/10531.jpg',
  'PetImages/Cat/2487.jpg',
  'PetImages/Cat/7817.jpg'],
 array([0, 1, 1, 1, 0, 1, 0, 0, 0, 0]))

In [19]:
# Augmentation
IMG_SIZE = 224
BATCH_SIZE = 224

# Basic Transformation
resize = tf.keras.Sequential([
    tf.keras.layers.experimental.preprocessing.Resizing(IMG_SIZE, BATCH_SIZE)
])

# DATA Augmentation
data_augemntation = tf.keras.Sequential([
    tf.keras.layers.experimental.preprocessing.RandomFlip("horizontal"),
    tf.keras.layers.experimental.preprocessing.RandomRotation(0.2),
    tf.keras.layers.experimental.preprocessing.RandomZoom(height_factor=(-0.3,-0.2))
])

In [20]:
# Function used to create a Tensorflow Data Object
AUTOTUNE = tf.data.experimental.AUTOTUNE
def get_dataset(paths , labels , train = True):
  image_paths = tf.convert_to_tensor(paths)
  labels = tf.convert_to_tensor(labels)

  image_dataset = tf.data.Dataset.from_tensor_slices(image_paths)
  label_dataset = tf.data.Dataset.from_tensor_slices(labels)
  
  dataset = tf.data.Dataset.zip((image_dataset , label_dataset))

  dataset = dataset.map(lambda image, label : load (image , label))
  dataset = dataset.map(lambda image, label: (resize(image), label), num_parallel_calls=2)
  dataset = dataset.shuffle(1000)
  dataset = dataset.batch(BATCH_SIZE)
  if train:
    dataset = dataset.map(lambda image, label: (data_augemntation(image), label), num_parallel_calls=2)
    dataset = dataset.repeat()
  return dataset

In [21]:
AUTOTUNE = tf.data.experimental.AUTOTUNE
print(AUTOTUNE)

-1


In [22]:
%time train_dataset = get_dataset(Train_paths, Train_labels)

image, label = next(iter(train_dataset))
print(image.shape)
print(label.shape)

CPU times: user 568 ms, sys: 31.4 ms, total: 599 ms
Wall time: 700 ms
(224, 224, 224, 3)
(224,)


In [23]:
%time val_dataset = get_dataset(Val_paths, Val_labels, train=False)

image, label = next(iter(val_dataset))
print(image.shape)
print(label.shape)

CPU times: user 22.4 ms, sys: 0 ns, total: 22.4 ms
Wall time: 23 ms
(224, 224, 224, 3)
(224,)


In [24]:
# Model part
# CNN architecture inspered by GoogleNets 2014
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Conv2D, MaxPooling2D, Flatten, Dropout, BatchNormalization, GlobalMaxPool2D

model = Sequential()

# Block 1
model.add(Conv2D(input_shape=(224, 224,3), padding="same", filters=32, kernel_size=(7, 7)))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.2))

# Block 2
model.add(Conv2D(filters=64, padding="valid", kernel_size=(5, 5)))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.2))

# Block 3
model.add(Conv2D(filters=128, padding="valid", kernel_size=(5, 5)))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.2))


# Block 4
model.add(Conv2D(filters=256, kernel_size=(3, 3)))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.2))

model.add(GlobalMaxPool2D())

model.add(Dense(units=256))
model.add(Activation('relu'))
model.add(Dropout(0.2))

model.add(Dense(units=1))
model.add(Activation('sigmoid'))


In [25]:
print(model.summary())

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 224, 224, 32)      4736      
                                                                 
 activation (Activation)     (None, 224, 224, 32)      0         
                                                                 
 batch_normalization (BatchN  (None, 224, 224, 32)     128       
 ormalization)                                                   
                                                                 
 max_pooling2d (MaxPooling2D  (None, 112, 112, 32)     0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 112, 112, 32)      0         
                                                                 
 conv2d_1 (Conv2D)           (None, 108, 108, 64)     

In [26]:
model.compile(
    loss='binary_crossentropy',
    optimizer='adam',
    metrics='accuracy'
)

In [27]:
# Train the model
history = model.fit(
    train_dataset,
    steps_per_epoch=len(Train_paths)// BATCH_SIZE,
    epochs=10,
    validation_data=val_dataset,
    validation_steps = len(Val_paths)//BATCH_SIZE
)

Epoch 1/10
83/83 [==============================] - 193s 2s/step - loss: 0.9856 - accuracy: 0.5692 - val_loss: 0.7206 - val_accuracy: 0.5620
Epoch 2/10
83/83 [==============================] - 173s 2s/step - loss: 0.6202 - accuracy: 0.6587 - val_loss: 0.6302 - val_accuracy: 0.6242
Epoch 3/10
83/83 [==============================] - 173s 2s/step - loss: 0.5867 - accuracy: 0.6907 - val_loss: 0.7281 - val_accuracy: 0.5332
Epoch 4/10
83/83 [==============================] - 170s 2s/step - loss: 0.5652 - accuracy: 0.7102 - val_loss: 0.8406 - val_accuracy: 0.5050
Epoch 5/10
83/83 [==============================] - 170s 2s/step - loss: 0.5531 - accuracy: 0.7178 - val_loss: 0.7931 - val_accuracy: 0.5144
Epoch 6/10
83/83 [==============================] - 169s 2s/step - loss: 0.5443 - accuracy: 0.7216 - val_loss: 0.7964 - val_accuracy: 0.5450
Epoch 7/10
83/83 [==============================] - 170s 2s/step - loss: 0.5282 - accuracy: 0.7372 - val_loss: 0.6993 - val_accuracy: 0.5599
Epoch 8/10
83